## Harjoitustyö

### Vaihe 1:

#### Hae Digitrafficin REST-rajapintapalvelusta (JSON-formaatissa) aktiiviset junat pääteosoitteella `/live-trains` Helsingin asemalta, joiden hakuparametrit ovat seuraavat:
* arrived_trains=10
* departed_trains=10
* include_nonstopping=False


#### Tämän jälkeen lue kyseinen data Pandasin dataframeen ilman muokkaamista, jolloin sarakkeiden otsikot ovat seuraavanlaiset:

 `cancelled`,
 `commuterLineID`,
 `departureDate`,
 `operatorShortCode`,
 `operatorUICCode`,
 `runningCurrently`,
 `timeTableRows`,
 `timetableAcceptanceDate`,
 `timetableType`,
 `trainCategory`,
 `trainNumber`,
 `trainType`,
 `version` 

In [23]:
import requests
import pandas as pd
import json
from pandas.io.json import json_normalize

#https://rata.digitraffic.fi/api/v1/live-trains/station/HKI?arrived_trains=10&departed_trains=10&include_nonstopping=false
station = "/station/HKI?"
arrived = "arrived_trains=10"
departed = "&departed_trains=10"
nonstopping = "&include_nonstopping=false"
url = "https://rata.digitraffic.fi/api/v1/live-trains{0}{1}{2}{3}".format(station, arrived, departed, nonstopping)

req = requests.get(url=url)

if req.status_code != 200:
    print(req.status_code)    
else:
    print(req.status_code)
    json_data = req.json()
    df = pd.read_json(json.dumps(json_data))    
    print(df)



200
    trainNumber departureDate  operatorUICCode operatorShortCode trainType  \
0          8122    2020-08-28               10                vr        HL   
1          8125    2020-08-28               10                vr        HL   
2          8127    2020-08-28               10                vr        HL   
3          8354    2020-08-28               10                vr        HL   
4          8357    2020-08-28               10                vr        HL   
5          8468    2020-08-28               10                vr        HL   
6          8473    2020-08-28               10                vr        HL   
7          8671    2020-08-28               10                vr        HL   
8          8673    2020-08-28               10                vr        HL   
9          8678    2020-08-28               10                vr        HL   
10         8679    2020-08-28               10                vr        HL   
11         8681    2020-08-28               10              

#### Poista kaikki muut sarakkeet paitsi `timeTableRows` ja `trainNumber`

In [24]:
    df = df[["timeTableRows", "trainNumber"]]
    print(df)

                                        timeTableRows  trainNumber
0   [{'stationShortCode': 'LPV', 'stationUICCode':...         8122
1   [{'stationShortCode': 'HKI', 'stationUICCode':...         8125
2   [{'stationShortCode': 'HKI', 'stationUICCode':...         8127
3   [{'stationShortCode': 'KLH', 'stationUICCode':...         8354
4   [{'stationShortCode': 'HKI', 'stationUICCode':...         8357
5   [{'stationShortCode': 'KKN', 'stationUICCode':...         8468
6   [{'stationShortCode': 'HKI', 'stationUICCode':...         8473
7   [{'stationShortCode': 'HKI', 'stationUICCode':...         8671
8   [{'stationShortCode': 'HKI', 'stationUICCode':...         8673
9   [{'stationShortCode': 'HKI', 'stationUICCode':...         8678
10  [{'stationShortCode': 'HKI', 'stationUICCode':...         8679
11  [{'stationShortCode': 'HKI', 'stationUICCode':...         8681
12  [{'stationShortCode': 'HKI', 'stationUICCode':...         9066
13  [{'stationShortCode': 'HKI', 'stationUICCode':...         

#### Tämän jälkeen pura `timeTableRows` sarakkeen arvot jokaisesta junasta erikseen (sarakkeen arvo jokaiselle junalle sisältää useampia JSON-objekteja) ja lisää dataan mukaan junan tiedot yksilöivä `trainNumber`

In [25]:

timeTableRowsList = df["timeTableRows"].tolist() 
df2 = json_normalize(timeTableRowsList[0])
trainNumbers = df['trainNumber'].astype(int)
df2.insert(0, 'trainNumber', trainNumbers)

print(df2)

    trainNumber stationShortCode  stationUICCode countryCode       type  \
0          8122              LPV              68          FI  DEPARTURE   
1          8125              MÄK             693          FI    ARRIVAL   
2          8127              MÄK             693          FI  DEPARTURE   
3          8354              PJM              69          FI    ARRIVAL   
4          8357              PJM              69          FI  DEPARTURE   
5          8468              VMO             847          FI    ARRIVAL   
6          8473              VMO             847          FI  DEPARTURE   
7          8671              HPL              72          FI    ARRIVAL   
8          8673              HPL              72          FI  DEPARTURE   
9          8678              KHK            1028          FI    ARRIVAL   
10         8679              KHK            1028          FI  DEPARTURE   
11         8681              ILA               9          FI    ARRIVAL   
12         9066          

#### Pudota sitten seuraavat sarakkeet datasta: `causes`, `estimateSource`, `liveEstimateTime`, `differenceInMinutes`, `commercialStop` ja `trainReady`

In [26]:
del_columns = ["causes","differenceInMinutes","commercialStop","trainReady.source","trainReady.accepted","trainReady.timestamp","liveEstimateTime","estimateSource"]
df2.drop(del_columns, axis=1, inplace=True)
print(df2)

    trainNumber stationShortCode  stationUICCode countryCode       type  \
0          8122              LPV              68          FI  DEPARTURE   
1          8125              MÄK             693          FI    ARRIVAL   
2          8127              MÄK             693          FI  DEPARTURE   
3          8354              PJM              69          FI    ARRIVAL   
4          8357              PJM              69          FI  DEPARTURE   
5          8468              VMO             847          FI    ARRIVAL   
6          8473              VMO             847          FI  DEPARTURE   
7          8671              HPL              72          FI    ARRIVAL   
8          8673              HPL              72          FI  DEPARTURE   
9          8678              KHK            1028          FI    ARRIVAL   
10         8679              KHK            1028          FI  DEPARTURE   
11         8681              ILA               9          FI    ARRIVAL   
12         9066          

#### Täytä jokaisen rivin sarakkeen `actualTime` puuttuvat arvot `scheduledTime` -sarakkeen arvolla.

In [27]:
df2['actualTime'].fillna(df2['scheduledTime'], inplace=True)
print(df2)

    trainNumber stationShortCode  stationUICCode countryCode       type  \
0          8122              LPV              68          FI  DEPARTURE   
1          8125              MÄK             693          FI    ARRIVAL   
2          8127              MÄK             693          FI  DEPARTURE   
3          8354              PJM              69          FI    ARRIVAL   
4          8357              PJM              69          FI  DEPARTURE   
5          8468              VMO             847          FI    ARRIVAL   
6          8473              VMO             847          FI  DEPARTURE   
7          8671              HPL              72          FI    ARRIVAL   
8          8673              HPL              72          FI  DEPARTURE   
9          8678              KHK            1028          FI    ARRIVAL   
10         8679              KHK            1028          FI  DEPARTURE   
11         8681              ILA               9          FI    ARRIVAL   
12         9066          

### Vaihe 2:

#### Pura tämän kansion sisällä olevan `data.html` -tiedoston `table`-elementin solujen arvot käyttäen regex-syntaksia ja yhdistä ne vaiheessa 1 luotuun dataframeen. Huom: tehtävässä tulee käyttää vain regexiä (Esim. Pandasin read_html()-metodin käyttö on kielletty)!

In [28]:
import re

html = open("data.html", "r").read()
#print(html.read())
tableText = re.search('<table>(.*?)</table>', html, re.M|re.I|re.S).group(0)

# remove white spaces and line changes etc.
tableText = re.sub(re.compile(r'\s'), '', tableText)

rows = re.findall(r'<tr(.*?)</tr>', tableText, re.M|re.I|re.S)
first = True
for row in rows:
    if first == True:
        first = False
        continue
    else:
        columns = re.findall(r'<td>(.*?)</td>', row, re.M|re.I|re.S)
        print(columns)
        df2 = df2.append({'trainNumber':columns[9], 'stationShortCode':columns[5], 'stationUICCode':columns[6], 'countryCode':columns[3], 
                          'type':columns[8], 'trainStopping':columns[7], 'commercialTrack':columns[2], 'cancelled':columns[1],
                          'scheduledTime':columns[4], 'actualTime':columns[0]}, ignore_index=True)

#titles = re.findall(r'<th>(.*?)</th>', tableText, re.M|re.I|re.S)
#print(titles)

#print(".....")
#print(df2.head(5))
#print(df2.tail(5))

['2019-08-06T04:48:00.000Z', 'False', '', 'FI', '2019-08-06T04:48:00.000Z', 'JVS', '1272', 'False', 'DEPARTURE', '150']
['2019-08-06T06:04:39.000Z', 'False', '', 'FI', '2019-08-06T06:04:39.000Z', 'R702', '98998', 'False', 'ARRIVAL', '150']
['2019-08-06T07:03:12.000Z', 'False', '', 'FI', '2019-08-06T07:03:12.000Z', 'ILA', '98998', '9', 'ARRIVAL', '8337']


### Vaihe 3:

#### Tee SQLite tietokanta ja lisää sinne vaiheessa 2 luotu dataframe.


In [29]:
import sqlite3
conn = sqlite3.connect('tkanta.db')
c = conn.cursor()

df2.to_sql('live_trains', conn, if_exists='replace')

query = '''SELECT * FROM live_trains'''
data = c.execute(query)
retrieved_data = data.fetchall()

# close connection
conn.close()

print(retrieved_data)

[(0, '8122', 'LPV', '68', 'FI', 'DEPARTURE', '1', '4', '0', '2020-08-28T11:46:00.000Z', '2020-08-28T11:46:09.000Z'), (1, '8125', 'MÄK', '693', 'FI', 'ARRIVAL', '1', '3', '0', '2020-08-28T11:47:30.000Z', '2020-08-28T11:48:04.000Z'), (2, '8127', 'MÄK', '693', 'FI', 'DEPARTURE', '1', '3', '0', '2020-08-28T11:48:00.000Z', '2020-08-28T11:48:30.000Z'), (3, '8354', 'PJM', '69', 'FI', 'ARRIVAL', '1', '3', '0', '2020-08-28T11:49:30.000Z', '2020-08-28T11:49:40.000Z'), (4, '8357', 'PJM', '69', 'FI', 'DEPARTURE', '1', '3', '0', '2020-08-28T11:50:00.000Z', '2020-08-28T11:50:07.000Z'), (5, '8468', 'VMO', '847', 'FI', 'ARRIVAL', '1', '3', '0', '2020-08-28T11:51:00.000Z', '2020-08-28T11:51:30.000Z'), (6, '8473', 'VMO', '847', 'FI', 'DEPARTURE', '1', '3', '0', '2020-08-28T11:51:30.000Z', '2020-08-28T11:51:53.000Z'), (7, '8671', 'HPL', '72', 'FI', 'ARRIVAL', '1', '3', '0', '2020-08-28T11:52:30.000Z', '2020-08-28T11:53:14.000Z'), (8, '8673', 'HPL', '72', 'FI', 'DEPARTURE', '1', '3', '0', '2020-08-28T11:5